In [1]:
import numpy as np
import pandas as pd
import swifter # Accelerator for pandas

from time import time
from unidecode import unidecode

import requests

In [2]:
# Converts the dataset with both year and month columns into a single column as dates
def month_year_to_date(df, year_name = "ano_exercicio", month_name = "mes_referencia"):
    '''
        Converte as informações separadas em colunas de mês e ano em uma única coluna com valores de datas.
    '''
    dates = {
        "year": df[year_name],
        "month": df[month_name],
        "day": np.repeat(15, df.shape[0])
    }
    df = df.copy()
    df = df.assign( data = pd.to_datetime(dates) )
    df = df.filter(["data", "ds_municipio", "ds_orgao", "ds_elemento", "id_despesa", "vl_despesa"])
    return df


# Converts the dataset with date column back to both year and month columns
def date_to_month_year(df, date_name = "data",):
    '''
        Converte as informações em uma única coluna com valores de datas em colunas de mês e ano.
    '''
    year = df.data.dt.year
    month = df.data.dt.month
    df = df.copy()
    df = df.assign( ano_exercicio = year, mes_referencia = month )
    df = df.filter(["ds_municipio", "ds_orgao", "ds_elemento", "id_despesa", "ano_exercicio", "mes_referencia", "vl_despesa"])
    return df

In [3]:
def quadrimestral_agg(df, year_index = "ano_exercicio", month_index = "mes_referencia"):
    '''
        Aggregate the data according to quadrimesters, summing the expense values in each timestamp.
    '''
    df_aux = df.copy()
    
    quad = np.repeat(0, month_df.shape[0])

    quad[ month_df.mes_referencia <= 4 ] = 1
    quad[ (month_df.mes_referencia > 4) & (month_df.mes_referencia <= 8) ] = 2
    quad[ month_df.mes_referencia > 8 ] = 3
    
    df_aux = df_aux.assign(quad = quad)

    df_aux = df_aux.groupby(["ds_municipio", "ds_orgao", "id_despesa", "ano", "quadrimester"]).agg({"vl_despesa": "sum"}).reset_index()
    
    # Converte as informações de ano e quadrimestre para o dia 15 do primeiro mês do quadrimestre correspondente
    date = df_aux.loc[:,["ano_exercicio", "quadrimester"]].swifter.progress_bar(False).apply(lambda x : datetime.date(int(x[0]), 4*int(x[1])-3, 15), axis = 1)
    df_aux = df_aux.assign(data = date)
    
    df_aux = df_aux.filter(["ds_municipio", "ds_orgao", "id_despesa", "data", "vl_despesa"])
    df_aux.data = pd.to_datetime(df_aux.data)
    
    return df_aux

In [4]:
df_month = pd.read_parquet("Data/month_data.parquet")
df_month = df_month.loc[df_month.ano_exercicio > 2010]

# Remove the observations of those expenses without description
df_month = df_month.loc[~df_month.ds_despesa.isna(), :]
df_month.head(4)

ano_exercicio  mes_referencia ds_municipio  \
722433         2011.0             1.0   adamantina   
722434         2011.0             1.0   adamantina   
722435         2011.0             1.0   adamantina   
722436         2011.0             1.0   adamantina   

                                  ds_orgao cod_despesa  \
722433  centro universitario de adamantina        0101   
722434  centro universitario de adamantina        0301   
722435  centro universitario de adamantina        1101   
722436  centro universitario de adamantina        1145   

                           ds_despesa              tp_orgao  \
722433      proventos - pessoal civil  centro universitario   
722434                  pessoal civil  centro universitario   
722435         vencimentos e salarios  centro universitario   
722436  ferias - abono constitucional  centro universitario   

             ds_funcao_governo                 ds_subfuncao_governo  \
722433      previdencia social             previdencia complementar   
722434      previdencia social             previdencia complementar   
722435  administracao,educacao  administracao geral,ensino superior   
722436           administracao                  administracao geral   

                                   ds_fonte_recurso      ds_modalidade_lic  \
722433  recursos proprios da administracao indireta  dispensa de licitacao   
722434  recursos proprios da administracao indireta  dispensa de licitacao   
722435  recursos proprios da administracao indireta  dispensa de licitacao   
722436  recursos proprios da administracao indireta  dispensa de licitacao   

        vl_despesa  
722433    13905.80  
722434     1207.24  
722435   674201.04  
722436    18427.02

In [5]:
quad = np.repeat(0, df_month.shape[0])

quad[ df_month.mes_referencia <= 4 ] = 1
quad[ (df_month.mes_referencia > 4) & (df_month.mes_referencia <= 8) ] = 2
quad[ df_month.mes_referencia > 8 ] = 3

df_month = df_month.assign(quad = quad)

def merge_list(lists):
    items = []
    for l in lists:
        items += l.split(",")
    return ",".join(np.unique(items))

start = time()
df_quad = df_month.groupby(["ds_municipio", "ds_orgao", "cod_despesa", "ano_exercicio", "quad"]).agg({
    "ds_despesa": "first",
    "vl_despesa": "sum",
    "tp_orgao": "first",
    "ds_funcao_governo": merge_list,
    "ds_subfuncao_governo": merge_list,
    "ds_fonte_recurso": merge_list,
    "ds_modalidade_lic": merge_list
}).reset_index()
print("Tempo de execução:", time() - start)

df_quad.head()

Tempo de execução: 393.4191372394562


ds_municipio                        ds_orgao cod_despesa  ano_exercicio  \
0   adamantina  camara municipal de adamantina        0101         2011.0   
1   adamantina  camara municipal de adamantina        0101         2011.0   
2   adamantina  camara municipal de adamantina        0101         2011.0   
3   adamantina  camara municipal de adamantina        0101         2012.0   
4   adamantina  camara municipal de adamantina        0101         2012.0   

   quad                 ds_despesa  vl_despesa          tp_orgao  \
0     1  proventos - pessoal civil    19768.56  camara municipal   
1     2  proventos - pessoal civil    18394.60  camara municipal   
2     3  proventos - pessoal civil    20826.12  camara municipal   
3     1  proventos - pessoal civil    24004.94  camara municipal   
4     2  proventos - pessoal civil    21337.72  camara municipal   

  ds_funcao_governo ds_subfuncao_governo ds_fonte_recurso  \
0       legislativa     acao legislativa          tesouro   
1       legislativa     acao legislativa          tesouro   
2       legislativa     acao legislativa          tesouro   
3       legislativa     acao legislativa          tesouro   
4       legislativa     acao legislativa          tesouro   

       ds_modalidade_lic  
0  dispensa de licitacao  
1  dispensa de licitacao  
2  dispensa de licitacao  
3   outros/nao aplicavel  
4   outros/nao aplicavel

Once everything is well sorted, we can proceed by deflating the prices along time. Clearly there is the influence of inflation on the prices, which can already be accounted for. So we remove this effect using the IPCA index. For that we implement a simple function to request the necessary information from the governmental statistics institute, IBGE. The API requires a specific month in order to provide the inflation index. As the entire analysis will be based on a single fixed point in time, it doesn't make a difference which month of the quadrimesters we use as reference.

In [12]:
def consulta_ipca(meses, anos, d = 2):
    '''
        Obtém os números-índice dos períodos especificados.
        args:
            meses: int/list
                Meses a serem consultados
            anos: int/list
                Anos a serem consultados (deve ter o mesmo tamanho de ``meses``)
            d: int
                Número de casas decimais a serem retornadas pela API
    '''
    if(type(meses) != list and type(meses) != np.ndarray): meses = [meses]
    if(type(anos) != list and type(anos) != np.ndarray): anos = [anos]
        
    # Criação da URL
    t = 1737
    v = 2266
    ps = []
    for mes, ano in zip(meses, anos):
        ps.append( "{}{}".format(str(ano), str(mes).zfill(2)) )
    p = ",".join(ps)
    url = "https://apisidra.ibge.gov.br/values/t/{}/n1/all/v/{}/p/{}/d/{}".format(t, v, p, d)

    # Requisição à API
    r = requests.get(url).json()
    vs = list(map(lambda x : float(x["V"]), r[1:]))
    
    if(len(meses) == 1): return vs[0]
    
    return vs

In [13]:
years = np.array([[i]*3 for i in range(2011, 2023)]).reshape(-1)
months = [4,8,12]*12
quads = [1,2,3]*12

ipcas = consulta_ipca(months, years, d = 2)
ipca_ref = ipcas[-1] # Last quadrimester of 2022

ipcas = pd.DataFrame({"ano_exercicio": years, "quad": quads, "ipca": ipcas})
ipcas.head()

ano_exercicio  quad     ipca
0           2011     1  3299.07
1           2011     2  3337.16
2           2011     3  3403.73
3           2012     1  3467.46
4           2012     2  3512.04

In [8]:
df_quad = df_quad.merge(ipcas, on = ["ano_exercicio", "quad"])
df_quad.head(2)

ds_municipio                        ds_orgao cod_despesa  ano_exercicio  \
0   adamantina  camara municipal de adamantina        0101         2011.0   
1   adamantina  camara municipal de adamantina        0301         2011.0   

   quad                 ds_despesa  vl_despesa          tp_orgao  \
0     1  proventos - pessoal civil    19768.56  camara municipal   
1     1              pessoal civil    22734.90  camara municipal   

  ds_funcao_governo ds_subfuncao_governo ds_fonte_recurso  \
0       legislativa     acao legislativa          tesouro   
1       legislativa     acao legislativa          tesouro   

       ds_modalidade_lic     ipca  
0  dispensa de licitacao  3299.07  
1  dispensa de licitacao  3299.07

In [9]:
df_quad["vl_despesa_defl"] = df_quad.vl_despesa * ipca_ref / df_quad.ipca
df_quad = df_quad.filter(['ds_municipio', 'ds_orgao', 'cod_despesa', 'ano_exercicio', 'quad',
                          'ds_despesa', 'tp_orgao', 'ds_funcao_governo', 'ds_subfuncao_governo',
                          'ds_fonte_recurso', 'ds_modalidade_lic', 'vl_despesa', 'vl_despesa_defl'])
df_quad.head()

ds_municipio                        ds_orgao cod_despesa  ano_exercicio  \
0   adamantina  camara municipal de adamantina        0101         2011.0   
1   adamantina  camara municipal de adamantina        0301         2011.0   
2   adamantina  camara municipal de adamantina        1101         2011.0   
3   adamantina  camara municipal de adamantina        1133         2011.0   
4   adamantina  camara municipal de adamantina        1137         2011.0   

   quad                             ds_despesa          tp_orgao  \
0     1              proventos - pessoal civil  camara municipal   
1     1                          pessoal civil  camara municipal   
2     1                 vencimentos e salarios  camara municipal   
3     1  gratificacao por exercicio de funcoes  camara municipal   
4     1      gratificacao por tempo de servico  camara municipal   

  ds_funcao_governo ds_subfuncao_governo ds_fonte_recurso  \
0       legislativa     acao legislativa          tesouro   
1       legislativa     acao legislativa          tesouro   
2       legislativa     acao legislativa          tesouro   
3       legislativa     acao legislativa          tesouro   
4       legislativa     acao legislativa          tesouro   

       ds_modalidade_lic  vl_despesa  vl_despesa_defl  
0  dispensa de licitacao    19768.56     38793.792375  
1  dispensa de licitacao    22734.90     44614.933524  
2  dispensa de licitacao    72753.62    142771.594330  
3  dispensa de licitacao      434.00       851.680946  
4  dispensa de licitacao     7253.17     14233.609886

In [32]:
df_quad.to_parquet("Data/quadrimestral_data.parquet", index = False)

As a way to make it all the same notation, let's reformat the files `rcl_municipios_deflated` and `info_municipios`, which have unformatted strings that could make the modeling difficult.

In [3]:
info_municipios = pd.read_excel("Data/info_municipios.xlsx")
rcl_municipios = pd.read_excel("Data/rcl_municipios_deflated.xlsx")

In [8]:
info_municipios["ds_municipio"] = info_municipios.ds_municipio.str.replace("-", " ").str.lower().apply(lambda x : unidecode(x))
info_municipios["atividade_principal"] = info_municipios.atividade_principal.str.replace("-", " ").str.lower().apply(lambda x : unidecode(x))

rcl_municipios["ds_municipio"] = rcl_municipios.ds_municipio.str.replace("-", " ").str.lower().apply(lambda x : unidecode(x))

In [14]:
info_municipios.to_excel("Data/info_municipios.xlsx", index = False)
rcl_municipios.to_excel("Data/rcl_municipios_deflated.xlsx", index = False)

## Year grouping

Let's also create a dataset in which the observations are aggregated by the whole year. In order to deflate the prices we can just use the IPCA values from the end of each year, which have already been obtained in the previous table.

In [8]:
df_year = df_month.groupby(["ds_municipio", "ds_orgao", "cod_despesa", "ano_exercicio"]).agg({
    "vl_despesa": "sum",
    "tp_orgao": "first"
}).reset_index()
df_year.head(4)

ds_municipio                        ds_orgao cod_despesa  ano_exercicio  \
0   adamantina  camara municipal de adamantina        0101         2011.0   
1   adamantina  camara municipal de adamantina        0101         2012.0   
2   adamantina  camara municipal de adamantina        0101         2013.0   
3   adamantina  camara municipal de adamantina        0101         2014.0   

   vl_despesa          tp_orgao  
0    58989.28  camara municipal  
1    69347.59  camara municipal  
2    73615.09  camara municipal  
3    80656.44  camara municipal

In [28]:
ipcas_year = ipcas[2::3]
ipcas_year = ipcas_year.iloc[:,[0,2]].reset_index(drop = True)
ipcas_year.head()

ano_exercicio     ipca
0           2011  3403.73
1           2012  3602.46
2           2013  3815.39
3           2014  4059.86
4           2015  4493.17

In [30]:
df_year = df_year.merge(ipcas_year, on = ["ano_exercicio"])
df_year.head(2)

ds_municipio                        ds_orgao cod_despesa  ano_exercicio  \
0   adamantina  camara municipal de adamantina        0101         2011.0   
1   adamantina  camara municipal de adamantina        0301         2011.0   

   vl_despesa          tp_orgao     ipca  
0    58989.28  camara municipal  3403.73  
1    67840.79  camara municipal  3403.73

In [32]:
df_year["vl_despesa_defl"] = df_year.vl_despesa * ipca_ref / df_year.ipca
df_year = df_year.filter(['ds_municipio', 'ds_orgao', 'cod_despesa', 'ano_exercicio', 'quad',
                          'ds_despesa', 'tp_orgao', 'vl_despesa', 'vl_despesa_defl'])
df_year.head()

ds_municipio                        ds_orgao cod_despesa  ano_exercicio  \
0   adamantina  camara municipal de adamantina        0101         2011.0   
1   adamantina  camara municipal de adamantina        0301         2011.0   
2   adamantina  camara municipal de adamantina        1101         2011.0   
3   adamantina  camara municipal de adamantina        1133         2011.0   
4   adamantina  camara municipal de adamantina        1137         2011.0   

           tp_orgao  vl_despesa  vl_despesa_defl  
0  camara municipal    58989.28    112200.999420  
1  camara municipal    67840.79    129037.079948  
2  camara municipal   225347.46    428623.814848  
3  camara municipal     1306.00      2484.087028  
4  camara municipal    22895.73     43548.993791

In [35]:
df_year.to_parquet("Data/year_data.parquet", index = False)